In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import math
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
df = pd.read_parquet('Resources/model_df.parquet')
df

,"('second_of_day',)","('day_of_year',)","('weekend',)","('holiday',)","('morning_rush',)","('evening rush',)","('prcp',)","('temp',)","('PUx',)","('PUy',)",...,"('PU_Bronx',)","('PU_Brooklyn',)","('PU_Manhattan',)","('PU_Queens',)","('PU_Staten Island',)","('DO_Bronx',)","('DO_Brooklyn',)","('DO_Manhattan',)","('DO_Queens',)","('DO_Staten Island',)"
0,5,1,False,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,...,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,...,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,...,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,...,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,False,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,...,True,False,False,False,False,True,False,False,False,False
6044443,85864,366,False,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,...,False,True,False,False,False,False,True,False,False,False
6044444,85870,366,False,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,...,False,False,True,False,False,False,False,True,False,False
6044445,86042,366,False,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,...,False,False,False,True,False,False,True,False,False,False


In [ ]:
df.columns = df.columns.str.replace("'", "")
df.columns = df.columns.str.replace("(", "")
df.columns = df.columns.str.replace(")", "")
df.columns = df.columns.str.replace(",", "")

In [ ]:
df

,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,...,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
0,5,1,False,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,...,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,...,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,...,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,...,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,False,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,...,True,False,False,False,False,True,False,False,False,False
6044443,85864,366,False,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,...,False,True,False,False,False,False,True,False,False,False
6044444,85870,366,False,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,...,False,False,True,False,False,False,False,True,False,False
6044445,86042,366,False,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,...,False,False,False,True,False,False,True,False,False,False


In [ ]:
df.dtypes

second_of_day         int32
day_of_year           int32
weekend                bool
holiday               int64
morning_rush          int32
evening rush          int32
prcp                float64
temp                float64
PUx                 float64
PUy                 float64
DOx                 float64
DOy                 float64
distance            float64
durationsec         float64
fare                float64
tolls               float64
airport             float64
congestion          float64
total               float64
class                 int64
PU_Bronx               bool
PU_Brooklyn            bool
PU_Manhattan           bool
PU_Queens              bool
PU_Staten Island       bool
DO_Bronx               bool
DO_Brooklyn            bool
DO_Manhattan           bool
DO_Queens              bool
DO_Staten Island       bool
dtype: object

In [ ]:
'''
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knr = KNeighborsRegressor(n_neighbors=k)
    knr.fit(X_train_scaled, y_train)
    train_score = knr.score(X_train_scaled, y_train)
    test_score = knr.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
plt.plot(range(1, 20, 2), train_scores, marker='o', label="training scores")
plt.plot(range(1, 20, 2), test_scores, marker="x", label="testing scores")
plt.xlabel("k neighbors")
plt.ylabel("accuracy score")
plt.legend()
plt.show()
'''

'\ntrain_scores = []\ntest_scores = []\nfor k in range(1, 20, 2):\n    knr = KNeighborsRegressor(n_neighbors=k)\n    knr.fit(X_train_scaled, y_train)\n    train_score = knr.score(X_train_scaled, y_train)\n    test_score = knr.score(X_test_scaled, y_test)\n    train_scores.append(train_score)\n    test_scores.append(test_score)\n    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")\n    \nplt.plot(range(1, 20, 2), train_scores, marker=\'o\', label="training scores")\nplt.plot(range(1, 20, 2), test_scores, marker="x", label="testing scores")\nplt.xlabel("k neighbors")\nplt.ylabel("accuracy score")\nplt.legend()\nplt.show()\n'

In [ ]:
df.columns

Index(['second_of_day', 'day_of_year', 'weekend', 'holiday', 'morning_rush',
       'evening rush', 'prcp', 'temp', 'PUx', 'PUy', 'DOx', 'DOy', 'distance',
       'durationsec', 'fare', 'tolls', 'airport', 'congestion', 'total',
       'class', 'PU_Bronx', 'PU_Brooklyn', 'PU_Manhattan', 'PU_Queens',
       'PU_Staten Island', 'DO_Bronx', 'DO_Brooklyn', 'DO_Manhattan',
       'DO_Queens', 'DO_Staten Island'],
      dtype='object')

In [ ]:
#change this range to filter days of data

df_filter_range = range(1, 366)

filter_mask = df['day_of_year'].isin(df_filter_range)

df_filtered = df[filter_mask]

df_filtered



,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,...,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
0,5,1,False,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,...,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,...,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,...,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,...,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,...,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6027274,86329,365,False,0,0,0,0.0,51.98,-8.237008e+06,4.972977e+06,...,False,False,True,False,False,False,False,True,False,False
6027275,86342,365,False,0,0,0,0.0,51.98,-8.231049e+06,4.985572e+06,...,False,False,True,False,False,True,False,False,False,False
6027276,86347,365,False,0,0,0,0.0,51.98,-8.235867e+06,4.973600e+06,...,False,False,True,False,False,False,True,False,False,False
6027277,86359,365,False,0,0,0,0.0,51.98,-8.228986e+06,4.967457e+06,...,False,True,False,False,False,False,True,False,False,False


In [ ]:
#change this to sample filter df

df_filtered_2 = df.sample(frac=0.20)

df_filtered_2

,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,...,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
956192,50414,60,False,0,0,0,0.0,35.06,-8.235573e+06,4.963843e+06,...,False,True,False,False,False,False,True,False,False,False
2312349,35821,118,True,0,0,0,0.0,46.04,-8.233149e+06,4.978027e+06,...,False,False,True,False,False,False,False,True,False,False
3714757,36506,226,False,0,0,0,0.0,68.00,-8.237101e+06,4.977923e+06,...,False,False,True,False,False,False,False,True,False,False
4771761,83017,292,False,0,0,0,0.0,64.94,-8.220685e+06,4.971340e+06,...,False,False,False,True,False,False,False,True,False,False
5931455,24110,359,False,0,0,0,0.0,30.92,-8.232189e+06,4.979764e+06,...,False,False,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544904,9049,35,True,0,0,0,0.0,39.02,-8.236629e+06,4.970973e+06,...,False,False,True,False,False,True,False,False,False,False
3789448,2367,231,True,0,0,0,0.0,73.04,-8.228703e+06,4.977225e+06,...,False,False,False,True,False,False,False,False,True,False
4285589,54513,263,False,0,0,0,0.0,80.06,-8.237633e+06,4.975305e+06,...,False,False,True,False,False,False,False,False,True,False
1031955,79325,62,True,0,0,0,3.9,53.60,-8.235586e+06,4.979010e+06,...,False,False,True,False,False,False,False,True,False,False


In [ ]:
df_filtered = df_filtered.drop(['fare', 'durationsec'], axis=1)

MemoryError: Unable to allocate 506. MiB for an array with shape (11, 6027279) and data type float64

In [ ]:
df_filtered_2 = df_filtered_2.drop(['fare','durationsec'], axis=1)

In [ ]:
X_2 = df_filtered_2.copy()
X_2 = X_2.drop('total', axis=1)

y_2 = df_filtered_2['total'].values.reshape(-1,1)

In [ ]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, random_state=29)

In [ ]:
#scaler2 = StandardScaler().fit(X_train_2)

#X_train_scaled2 = scaler2.transform(X_train_2)
#X_test_scaled2 = scaler2.transform(X_test_2)

#X_test_scaled2

array([[-0.83156403,  1.00042456, -0.66112114, ..., -1.07262568,
         2.19049   , -0.10165821],
       [ 1.25960513, -0.96740137, -0.66112114, ..., -1.07262568,
        -0.45651886, -0.10165821],
       [ 0.59450827,  0.57743394, -0.66112114, ..., -1.07262568,
         2.19049   , -0.10165821],
       ...,
       [ 0.76660368, -0.68234247, -0.66112114, ...,  0.93229168,
        -0.45651886, -0.10165821],
       [ 0.35512111,  1.0831836 , -0.66112114, ...,  0.93229168,
        -0.45651886, -0.10165821],
       [-1.21242731, -0.90303323, -0.66112114, ..., -1.07262568,
        -0.45651886, -0.10165821]])

In [ ]:
'''
knr2 = KNeighborsRegressor(n_neighbors=5)
knr2.fit(X_train_scaled2, y_train_2)
print(knr2.score(X_train_scaled2, y_train_2))
print(knr2.score(X_test_scaled2, y_test_2))

#target = total
#removed - 'fare', 'durationsec'
'''

"\nknr2 = KNeighborsRegressor(n_neighbors=5)\nknr2.fit(X_train_scaled2, y_train_2)\nprint(knr2.score(X_train_scaled2, y_train_2))\nprint(knr2.score(X_test_scaled2, y_test_2))\n\n#target = total\n#removed - 'fare', 'durationsec'\n"

In [ ]:
'''
y_predicted = knr2.predict(X_test_scaled2)

mae = mean_absolute_error(y_test_2, y_predicted)

print(mae)
'''

'\ny_predicted = knr2.predict(X_test_scaled2)\n\nmae = mean_absolute_error(y_test_2, y_predicted)\n\nprint(mae)\n'

In [ ]:
uber_df = df_filtered[(df_filtered['class'] == 1) & (df['total'] > 0)]
taxi_df = df_filtered[(df_filtered['class'] == 0) & (df['total'] > 0)]
lyft_df = df_filtered[(df_filtered['class'] == 2) & (df['total'] > 0)]

C:\Users\Nik\AppData\Local\Temp\ipykernel_54836\1267027460.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  uber_df = df_filtered[(df_filtered['class'] == 1) & (df['total'] > 0)]
C:\Users\Nik\AppData\Local\Temp\ipykernel_54836\1267027460.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  taxi_df = df_filtered[(df_filtered['class'] == 0) & (df['total'] > 0)]
C:\Users\Nik\AppData\Local\Temp\ipykernel_54836\1267027460.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lyft_df = df_filtered[(df_filtered['class'] == 2) & (df['total'] > 0)]


In [ ]:
uber_df

,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,...,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,...,True,False,False,False,False,True,False,False,False,False
8,75,1,False,1,0,0,0.0,42.80,-8.221833e+06,4.988439e+06,...,True,False,False,False,False,True,False,False,False,False
13,162,1,False,1,0,0,0.0,42.80,-8.236580e+06,4.974117e+06,...,False,False,True,False,False,False,False,True,False,False
19,202,1,False,1,0,0,0.0,42.80,-8.236394e+06,4.976052e+06,...,False,False,True,False,False,False,False,True,False,False
27,257,1,False,1,0,0,0.0,42.80,-8.236601e+06,4.964303e+06,...,False,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6027269,86286,365,False,0,0,0,0.0,51.98,-8.237299e+06,4.971260e+06,...,False,False,True,False,False,False,True,False,False,False
6027271,86310,365,False,0,0,0,0.0,51.98,-8.231792e+06,4.962136e+06,...,False,True,False,False,False,False,False,False,True,False
6027273,86323,365,False,0,0,0,0.0,51.98,-8.236512e+06,4.977337e+06,...,False,False,True,False,False,True,False,False,False,False
6027274,86329,365,False,0,0,0,0.0,51.98,-8.237008e+06,4.972977e+06,...,False,False,True,False,False,False,False,True,False,False


In [ ]:
X = uber_df.copy()
X = X.drop('total', axis=1)

y = uber_df['total'].values.reshape(-1,1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=29)

In [ ]:
#scaler = StandardScaler().fit(X_train)

#X_train_scaled = scaler.transform(X_train)
#X_test_scaled = scaler.transform(X_test)

In [ ]:
knr = KNeighborsRegressor(n_neighbors=5)
knr.fit(X_train_2, y_train_2)
print('Train/Test r-2:')
print(knr.score(X_train_2, y_train_2))
print(knr.score(X_test_2, y_test_2))

In [ ]:
y_predicted = knr.predict(X_test)

mae = mean_absolute_error(y_test, y_predicted)
print('Mean abs Error:')
print(mae)

In [ ]:
knr = KNeighborsRegressor(n_neighbors=5)
knr.fit(X_train_scaled, y_train)
print('Uber Train/Test r-2:')
print(knr.score(X_train_scaled, y_train))
print(knr.score(X_test_scaled, y_test))

Uber Train/Test r-2:
0.8749020765928646
0.8113413713664941


In [68]:
y_predicted = knr.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_predicted)
print('Uber Mean abs Error:')
print(mae)

Uber Mean abs Error:
5.206614805381102


In [69]:
X = lyft_df.copy()
X = X.drop('total', axis=1)

y = lyft_df['total'].values.reshape(-1,1)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=29)

In [71]:
scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [72]:
knr = KNeighborsRegressor(n_neighbors=5)
knr.fit(X_train_scaled, y_train)
print('Lyft Train/Test r-2:')
print(knr.score(X_train_scaled, y_train))
print(knr.score(X_test_scaled, y_test))

Lyft Train/Test r-2:
0.9210767695840043
0.8804085060290018


In [73]:
y_predicted = knr.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_predicted)
print('Lyft Mean abs Error:')
print(mae)

Lyft Mean abs Error:
3.8264940219315124


In [74]:
X = taxi_df.copy()
X = X.drop('total', axis=1)

y = taxi_df['total'].values.reshape(-1,1)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=29)

In [76]:
scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [77]:
knr = KNeighborsRegressor(n_neighbors=5)
knr.fit(X_train_scaled, y_train)
print('Taxi Train/Test r-2:')
print(knr.score(X_train_scaled, y_train))
print(knr.score(X_test_scaled, y_test))

Taxi Train/Test r-2:
0.9265903342018228
0.8884790161202938


In [78]:
y_predicted = knr.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_predicted)
print('Taxi Mean abs Error:')
print(mae)

Taxi Mean abs Error:
3.879458246939178


In [10]:
knr_test = KNeighborsRegressor(n_neighbors=5)

In [ ]:
param_grid = {
    'leaf_size': [15, 30, 50, 100]
}
grid_clf = GridSearchCV(knr_test, param_grid, verbose=3)
grid_clf.fit(X_train_scaled2, y_train_2)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
